## Examples using intake with xarray

In this notebook, we'll take a look at some of the file formats that can be written to `xarray` objects.

In [ ]:
import intake
intake.output_notebook()

## Inline example

Anything that can be opened with `xarray.open_dataset` can be accessed with intake using `intake.open_netcdf`. 

### GRIB
In this first example we will use an OpenDAP URL to lazily access a grib file. 

In [ ]:
grib_da = intake.open_netcdf('http://thredds.ucar.edu/thredds/dodsC/grib/FNMOC/WW3/Global_1p0deg/Best', chunks={}).to_dask()
grib_da

### Image

Intake-xarray also provides easy access to other backends to support loading other file formats. Such as image data.

In [ ]:
image = intake.open_xarray_image('https://s3.amazonaws.com/earth-data/UCMerced_LandUse/Images/tenniscourt/tenniscourt02.tif')
image_da = image.to_dask()
image_da

In [ ]:
from PIL import Image

In [ ]:
Image.fromarray(image_da.data.compute())

#### Multiple Images 

We can lazily load lots of images from s3 and other backends using glob notation. 

In [ ]:
multi_image = intake.open_xarray_image('s3://earth-data/UCMerced_LandUse/Images/*/*05.tif', 
                                       storage_options=dict(anon=True))
multi_image_da = multi_image.to_dask()
multi_image_da

In [ ]:
np_array = multi_image_da[1].dropna('x', 'all').dropna('y', 'all').data.compute().astype('uint8')
Image.fromarray(np_array)

## Catalog examples

There is a sample catalog at `./catalog.yml` containing some examples of how to use the intake-xarray plugins. 

In [ ]:
cat = intake.open_catalog('catalog.yml')
list(cat)

### Grib: from unauthenticated OpenDAP

Grib files can be read from catalog specs just as in the inline example above.

In [ ]:
grib_thredds = cat.grib_thredds
grib_thredds_da = grib_thredds.to_dask()
grib_thredds_da

In [ ]:
import hvplot.xarray

In [ ]:
var = 'sig_wav_ht_surface'
time_dim = grib_thredds_da[var].dims[0]
grib_thredds_da[var].hvplot(x='lon', y='lat', groupby=time_dim, rasterize=True)

## Image

Image files can be read in using a variety of backends.

In [ ]:
image = cat.image(landuse='airplane', id=0)
image_da = image.to_dask()
image_da

In [ ]:
np_array = image_da.data.compute()
Image.fromarray(np_array)

#### Multiple Images

When loading multiple images, if the images don't have exactly the same size, you will have nan slices in your data and the dtype will be floats. 

In [ ]:
%time labelled_da = cat.images_labelled(id=0).to_dask()
labelled_da

In [ ]:
np_array = (labelled_da.sel(landuse='beach', id=0)
                       .dropna('x', 'all')
                       .dropna('y', 'all')
                       .data.compute().astype('uint8'))
Image.fromarray(np_array)

If we aren't interested in labels, then it is even faster to read in the files. 

In [ ]:
%time unlabelled_da = cat.images_unlabelled(id=5).to_dask()
unlabelled_da

In [ ]:
np_array = unlabelled_da[1].dropna('x', 'all').dropna('y', 'all').data.compute().astype('uint8')
Image.fromarray(np_array)

### GeoTiff
For geotiffs there is a special `rasterio` backend that support loading using gdal. 

In [ ]:
geotiff = cat.geotiff
geotiff_da = geotiff.to_dask()
geotiff_da

In [ ]:
geotiff.plot.band_image()